# Phase 5: REST API Layer Demo

This notebook demonstrates the FastAPI REST API layer from Phase 5 of the IBKR Gateway project.

**Prerequisites:**
- IBKR Gateway or TWS running in paper trading mode
- API server running: `uvicorn api.server:app --reload`
- Default API URL: `http://127.0.0.1:8000`

## Features Demonstrated:
1. **Health Check** - Server status and IBKR connection
2. **Market Data** - Quotes and historical bars via HTTP
3. **Account Data** - Summary, positions, P&L
4. **Order Management** - Preview, place, cancel orders
5. **Authentication** - API key header
6. **Error Handling** - Consistent error responses

## API Documentation:
- OpenAPI/Swagger: http://127.0.0.1:8000/docs
- ReDoc: http://127.0.0.1:8000/redoc

In [1]:
import requests
from pprint import pprint

# API base URL
BASE_URL = "http://127.0.0.1:8000"

# Optional: API key (if server has API_KEY env var set)
API_KEY = None  # Set to your key if authentication is enabled

def get_headers():
    """Get request headers with optional API key."""
    headers = {"Content-Type": "application/json"}
    if API_KEY:
        headers["X-API-Key"] = API_KEY
    return headers

def pretty_print(response):
    """Pretty print JSON response."""
    print(f"Status: {response.status_code}")
    try:
        pprint(response.json())
    except:
        print(response.text)

## 1. Health Check

The `/health` endpoint shows server status and IBKR connection state.
- Returns `status: "ok"` when IBKR is connected
- Returns `status: "degraded"` when IBKR is not connected
- Does NOT require authentication

In [2]:
print("GET /health")
print("=" * 50)

response = requests.get(f"{BASE_URL}/health", timeout=10)
pretty_print(response)

if response.status_code == 200:
    data = response.json()
    print(f"\nIBKR Connected: {data['ibkr_connected']}")
    print(f"Trading Mode:   {data['trading_mode']}")
    print(f"Orders Enabled: {data['orders_enabled']}")

GET /health
Status: 200
{'ibkr_connected': True,
 'orders_enabled': True,
 'server_time': None,
 'status': 'ok',
 'trading_mode': 'paper',
 'version': '0.1.0'}

IBKR Connected: True
Trading Mode:   paper
Orders Enabled: True


## 2. Market Data - Quotes

Get real-time quotes for instruments via `POST /market-data/quote`.

In [3]:
print("POST /market-data/quote - AAPL Stock")
print("=" * 50)

quote_request = {
    "symbol": "AAPL",
    "securityType": "STK",
    "exchange": "SMART",
    "currency": "USD"
}

response = requests.post(
    f"{BASE_URL}/market-data/quote",
    headers=get_headers(),
    json=quote_request
)
pretty_print(response)

if response.status_code == 200:
    quote = response.json()
    print("\nAAPL Quote Summary:")
    print(f"  Bid:  ${quote['bid']:.2f} x {quote['bidSize']}")
    print(f"  Ask:  ${quote['ask']:.2f} x {quote['askSize']}")
    print(f"  Last: ${quote['last']:.2f}")

POST /market-data/quote - AAPL Stock
Status: 200
{'ask': 270.98,
 'askSize': 100.0,
 'bid': 270.79,
 'bidSize': 100.0,
 'conId': 265598,
 'last': 270.88,
 'lastSize': 200.0,
 'source': 'IBKR_SNAPSHOT',
 'symbol': 'AAPL',
 'timestamp': '2025-12-23T01:08:07.527819Z',
 'volume': 7.0}

AAPL Quote Summary:
  Bid:  $270.79 x 100.0
  Ask:  $270.98 x 100.0
  Last: $270.88


In [4]:
print("POST /market-data/quote - MES Futures")
print("=" * 50)

quote_request = {
    "symbol": "MES",
    "securityType": "FUT",
    "exchange": "CME",
    "currency": "USD"
}

response = requests.post(
    f"{BASE_URL}/market-data/quote",
    headers=get_headers(),
    json=quote_request
)
pretty_print(response)

POST /market-data/quote - MES Futures
Status: 200
{'ask': 6932.75,
 'askSize': 13.0,
 'bid': 6932.5,
 'bidSize': 30.0,
 'conId': 750150186,
 'last': 6932.5,
 'lastSize': 1.0,
 'source': 'IBKR_SNAPSHOT',
 'symbol': 'MES',
 'timestamp': '2025-12-23T01:08:15.817563Z',
 'volume': 10472.0}


## 3. Market Data - Historical Bars

Get historical OHLCV bars via `POST /market-data/historical`.

In [5]:
print("POST /market-data/historical - AAPL 5 Days")
print("=" * 50)

historical_request = {
    "symbol": "AAPL",
    "securityType": "STK",
    "exchange": "SMART",
    "currency": "USD",
    "barSize": "1d",
    "duration": "5d",
    "whatToShow": "TRADES",
    "rthOnly": True
}

response = requests.post(
    f"{BASE_URL}/market-data/historical",
    headers=get_headers(),
    json=historical_request
)

if response.status_code == 200:
    data = response.json()
    print(f"Symbol: {data['symbol']}")
    print(f"Bar Count: {data['barCount']}")
    print("\nBars:")
    print(f"{'Date':<12} {'Open':>10} {'High':>10} {'Low':>10} {'Close':>10} {'Volume':>12}")
    print("-" * 70)
    for bar in data['bars']:
        date = bar['time'][:10]
        print(f"{date:<12} {bar['open']:>10.2f} {bar['high']:>10.2f} {bar['low']:>10.2f} {bar['close']:>10.2f} {bar['volume']:>12,.0f}")
else:
    pretty_print(response)

POST /market-data/historical - AAPL 5 Days
Symbol: AAPL
Bar Count: 5

Bars:
Date               Open       High        Low      Close       Volume
----------------------------------------------------------------------
2025-12-16       272.70     275.50     271.79     274.61   17,191,726
2025-12-17       275.00     276.16     271.64     271.84   20,970,018
2025-12-18       273.61     273.63     266.95     272.19   25,722,115
2025-12-19       272.39     274.60     269.90     273.67   39,565,798
2025-12-22       272.88     273.88     270.50     270.97   20,316,150


## 4. Account Data - Summary

Get account summary via `GET /account/summary`.

In [6]:
print("GET /account/summary")
print("=" * 50)

response = requests.get(
    f"{BASE_URL}/account/summary",
    headers=get_headers()
)

if response.status_code == 200:
    summary = response.json()
    print(f"Account:           {summary['accountId']}")
    print(f"Currency:          {summary['currency']}")
    print(f"Net Liquidation:   ${summary['netLiquidation']:,.2f}")
    print(f"Cash:              ${summary['cash']:,.2f}")
    print(f"Buying Power:      ${summary['buyingPower']:,.2f}")
    print(f"Margin Excess:     ${summary['marginExcess']:,.2f}")
else:
    pretty_print(response)

GET /account/summary
Account:           DUM096342
Currency:          CAD
Net Liquidation:   $1,012,087.44
Cash:              $1,010,552.12
Buying Power:      $3,372,805.99
Margin Excess:     $1,011,864.13


## 5. Account Data - Positions

Get open positions via `GET /account/positions`.

In [7]:
print("GET /account/positions")
print("=" * 50)

response = requests.get(
    f"{BASE_URL}/account/positions",
    headers=get_headers()
)

if response.status_code == 200:
    data = response.json()
    print(f"Account: {data['accountId']}")
    print(f"Position Count: {data['positionCount']}")
    
    if data['positions']:
        print(f"\n{'Symbol':<12} {'Qty':>8} {'Avg Price':>12} {'Mkt Price':>12} {'Unrealized':>12}")
        print("-" * 60)
        for pos in data['positions']:
            print(f"{pos['symbol']:<12} {pos['quantity']:>8.0f} {pos['avgPrice']:>12.2f} {pos['marketPrice']:>12.2f} {pos['unrealizedPnl']:>12.2f}")
    else:
        print("\nNo open positions.")
else:
    pretty_print(response)

GET /account/positions
Account: DUM096342
Position Count: 1

Symbol            Qty    Avg Price    Mkt Price   Unrealized
------------------------------------------------------------
AAPL                2       101.06       270.77       137.30


## 6. Account Data - P&L

Get profit & loss via `GET /account/pnl`.

In [8]:
print("GET /account/pnl")
print("=" * 50)

response = requests.get(
    f"{BASE_URL}/account/pnl",
    headers=get_headers()
)

if response.status_code == 200:
    pnl = response.json()
    print(f"Account:    {pnl['accountId']}")
    print(f"Timeframe:  {pnl['timeframe']}")
    print(f"Realized:   ${pnl['realized']:,.2f}")
    print(f"Unrealized: ${pnl['unrealized']:,.2f}")
    
    if pnl.get('bySymbol'):
        print("\nBy Symbol:")
        for symbol, detail in pnl['bySymbol'].items():
            print(f"  {symbol}: R=${detail['realized']:,.2f}, U=${detail['unrealized']:,.2f}")
else:
    pretty_print(response)

GET /account/pnl
Account:    DUM096342
Timeframe:  CURRENT
Realized:   $0.00
Unrealized: $137.30

By Symbol:
  AAPL: R=$0.00, U=$137.30


## 7. Order Preview

Preview an order before placing via `POST /orders/preview`.

In [9]:
print("POST /orders/preview - AAPL Limit Order")
print("=" * 50)

preview_request = {
    "instrument": {
        "symbol": "AAPL",
        "securityType": "STK",
        "exchange": "SMART",
        "currency": "USD"
    },
    "side": "BUY",
    "quantity": 10,
    "orderType": "LMT",
    "limitPrice": 150.00,
    "tif": "DAY"
}

response = requests.post(
    f"{BASE_URL}/orders/preview",
    headers=get_headers(),
    json=preview_request
)

if response.status_code == 200:
    preview = response.json()
    print(f"Estimated Price:    ${preview.get('estimatedPrice', 0):,.2f}")
    print(f"Estimated Notional: ${preview.get('estimatedNotional', 0):,.2f}")
    if preview.get('warnings'):
        print(f"Warnings: {preview['warnings']}")
else:
    pretty_print(response)

POST /orders/preview - AAPL Limit Order
Estimated Price:    $150.00
Estimated Notional: $1,500.00
Warnings: ['Preview is simulated using current market data']


## 8. Place Order

Place an order via `POST /orders`.

**Safety Note:** When `ORDERS_ENABLED=false` (default), orders return `SIMULATED` status.

In [10]:
print("POST /orders - Place Far-From-Market Order")
print("=" * 50)

order_request = {
    "instrument": {
        "symbol": "AAPL",
        "securityType": "STK",
        "exchange": "SMART",
        "currency": "USD"
    },
    "side": "BUY",
    "quantity": 1,
    "orderType": "LMT",
    "limitPrice": 1.00,  # Far from market - won't fill
    "tif": "DAY"
}

response = requests.post(
    f"{BASE_URL}/orders",
    headers=get_headers(),
    json=order_request
)

if response.status_code == 200:
    result = response.json()
    print(f"Status:   {result['status']}")
    print(f"Order ID: {result.get('orderId')}")
    
    if result['status'] == 'SIMULATED':
        print("\n[SAFE] Order was simulated - ORDERS_ENABLED=false")
        print(f"Errors: {result.get('errors', [])}")
    elif result['status'] == 'ACCEPTED':
        print("\n[LIVE] Order was sent to IBKR!")
        order_id = result['orderId']
else:
    pretty_print(response)
    order_id = None

POST /orders - Place Far-From-Market Order
Status:   ACCEPTED
Order ID: 13

[LIVE] Order was sent to IBKR!


## 9. Order Status

Get order status via `GET /orders/{order_id}/status`.

In [11]:
# Use order_id from previous cell, or set manually
order_id_to_check = order_id if 'order_id' in dir() and order_id else "12345"

print(f"GET /orders/{order_id_to_check}/status")
print("=" * 50)

response = requests.get(
    f"{BASE_URL}/orders/{order_id_to_check}/status",
    headers=get_headers()
)

if response.status_code == 200:
    status = response.json()
    print(f"Order ID:  {status['orderId']}")
    print(f"Status:    {status['status']}")
    print(f"Filled:    {status['filledQuantity']}")
    print(f"Remaining: {status['remainingQuantity']}")
    print(f"Avg Price: ${status['avgFillPrice']:.2f}")
else:
    pretty_print(response)

GET /orders/13/status
Order ID:  13
Status:    SUBMITTED
Filled:    0.0
Remaining: 1.0
Avg Price: $0.00


## 10. Open Orders

List all open orders via `GET /orders/open`.

In [12]:
print("GET /orders/open")
print("=" * 50)

response = requests.get(
    f"{BASE_URL}/orders/open",
    headers=get_headers()
)

if response.status_code == 200:
    orders = response.json()
    if orders:
        print(f"{'Order ID':<10} {'Symbol':<10} {'Side':<6} {'Qty':>6} {'Type':<8} {'Status':<12}")
        print("-" * 60)
        for order in orders:
            print(f"{order['order_id']:<10} {order['symbol']:<10} {order['side']:<6} {order['quantity']:>6} {order['order_type']:<8} {order['status']:<12}")
    else:
        print("No open orders.")
else:
    pretty_print(response)

GET /orders/open
Order ID   Symbol     Side      Qty Type     Status      
------------------------------------------------------------
844753850  AAPL       BUY       1.0 LMT      PreSubmitted


## 11. Cancel Order

Cancel an order via `POST /orders/{order_id}/cancel`.

In [13]:
order_id_to_cancel = order_id if 'order_id' in dir() and order_id else "12345"

print(f"POST /orders/{order_id_to_cancel}/cancel")
print("=" * 50)

response = requests.post(
    f"{BASE_URL}/orders/{order_id_to_cancel}/cancel",
    headers=get_headers()
)

if response.status_code == 200:
    result = response.json()
    print(f"Order ID: {result['orderId']}")
    print(f"Status:   {result['status']}")
    print(f"Message:  {result.get('message', '')}")
else:
    pretty_print(response)

POST /orders/13/cancel
Order ID: 13
Status:   CANCELLED
Message:  Order cancelled (IBKR status: Cancelled)


## 12. Error Handling

The API returns consistent error responses with error codes.

In [14]:
print("Validation Error - Invalid Order Side")
print("=" * 50)

invalid_request = {
    "instrument": {
        "symbol": "AAPL",
        "securityType": "STK"
    },
    "side": "INVALID",  # Invalid!
    "quantity": 10,
    "orderType": "LMT",
    "limitPrice": 150.00
}

response = requests.post(
    f"{BASE_URL}/orders/preview",
    headers=get_headers(),
    json=invalid_request
)

print(f"Status Code: {response.status_code}")
pretty_print(response)

Validation Error - Invalid Order Side
Status Code: 422
Status: 422
{'detail': [{'ctx': {'error': {}},
             'input': 'INVALID',
             'loc': ['body', 'side'],
             'msg': "Value error, side must be 'BUY' or 'SELL', got INVALID",
             'type': 'value_error'}]}


In [15]:
print("Validation Error - Missing Required Field")
print("=" * 50)

invalid_request = {
    "symbol": "AAPL",
    # Missing securityType!
}

response = requests.post(
    f"{BASE_URL}/market-data/quote",
    headers=get_headers(),
    json=invalid_request
)

print(f"Status Code: {response.status_code}")
pretty_print(response)

Validation Error - Missing Required Field
Status Code: 422
Status: 422
{'detail': [{'input': {'symbol': 'AAPL'},
             'loc': ['body', 'securityType'],
             'msg': 'Field required',
             'type': 'missing'}]}


## 13. Authentication

When `API_KEY` environment variable is set on the server, requests require the `X-API-Key` header.

In [16]:
print("Authentication Demo")
print("=" * 50)
print("""
To enable authentication:
1. Set API_KEY env var before starting server:
   export API_KEY=your-secret-key
   uvicorn api.server:app --reload

2. Include X-API-Key header in requests:
   headers = {"X-API-Key": "your-secret-key"}

3. Without valid key, you'll get 401 Unauthorized:
   {
       "detail": {
           "error_code": "UNAUTHORIZED",
           "message": "Invalid or missing API key"
       }
   }
""")

# Test without API key
print("Request without X-API-Key header:")
response = requests.get(f"{BASE_URL}/health")
print(f"Health (no auth required): {response.status_code}")

Authentication Demo

To enable authentication:
1. Set API_KEY env var before starting server:
   export API_KEY=your-secret-key
   uvicorn api.server:app --reload

2. Include X-API-Key header in requests:
   headers = {"X-API-Key": "your-secret-key"}

3. Without valid key, you'll get 401 Unauthorized:
   {
       "detail": {
           "error_code": "UNAUTHORIZED",
           "message": "Invalid or missing API key"
       }
   }

Request without X-API-Key header:
Health (no auth required): 200


## 14. Error Codes Reference

The API uses consistent error codes across all endpoints.

In [17]:
print("API Error Codes Reference")
print("=" * 60)
print("""
Connection Errors:
  IBKR_CONNECTION_ERROR  - Cannot connect to IBKR gateway
  IBKR_TIMEOUT           - IBKR operation timed out

Market Data Errors:
  MARKET_DATA_PERMISSION_DENIED - No permission for data
  NO_DATA                       - No data available
  PACING_VIOLATION              - Too many requests

Contract Errors:
  CONTRACT_RESOLUTION_ERROR - Cannot resolve contract

Account Errors:
  ACCOUNT_ERROR - Account operation failed

Order Errors:
  TRADING_DISABLED      - Orders disabled (safety)
  ORDER_VALIDATION_ERROR - Invalid order parameters
  ORDER_PLACEMENT_ERROR  - Order placement failed
  ORDER_NOT_FOUND        - Order not found
  ORDER_CANCEL_ERROR     - Cancel operation failed

General Errors:
  UNAUTHORIZED     - Missing/invalid API key (401)
  VALIDATION_ERROR - Request validation failed (422)
  INTERNAL_ERROR   - Unexpected server error (500)
  TIMEOUT          - Request timeout (504)
""")

API Error Codes Reference

Connection Errors:
  IBKR_CONNECTION_ERROR  - Cannot connect to IBKR gateway
  IBKR_TIMEOUT           - IBKR operation timed out

Market Data Errors:
  MARKET_DATA_PERMISSION_DENIED - No permission for data
  NO_DATA                       - No data available
  PACING_VIOLATION              - Too many requests

Contract Errors:
  CONTRACT_RESOLUTION_ERROR - Cannot resolve contract

Account Errors:
  ACCOUNT_ERROR - Account operation failed

Order Errors:
  TRADING_DISABLED      - Orders disabled (safety)
  ORDER_VALIDATION_ERROR - Invalid order parameters
  ORDER_PLACEMENT_ERROR  - Order placement failed
  ORDER_NOT_FOUND        - Order not found
  ORDER_CANCEL_ERROR     - Cancel operation failed

General Errors:
  UNAUTHORIZED     - Missing/invalid API key (401)
  VALIDATION_ERROR - Request validation failed (422)
  INTERNAL_ERROR   - Unexpected server error (500)
  TIMEOUT          - Request timeout (504)



## 15. API Endpoints Summary

In [18]:
print("API Endpoints Summary")
print("=" * 60)
print("""
Health:
  GET  /health              - Server and IBKR status

Market Data:
  POST /market-data/quote      - Get quote for instrument
  POST /market-data/historical - Get historical bars

Account:
  GET  /account/summary     - Account summary (NLV, margin)
  GET  /account/positions   - Open positions
  GET  /account/pnl         - Profit & loss

Orders:
  POST /orders/preview           - Preview order impact
  POST /orders                   - Place order
  GET  /orders/open              - List open orders
  GET  /orders/{id}/status       - Get order status
  POST /orders/{id}/cancel       - Cancel order

Documentation:
  GET  /docs    - Swagger UI
  GET  /redoc   - ReDoc
""")

API Endpoints Summary

Health:
  GET  /health              - Server and IBKR status

Market Data:
  POST /market-data/quote      - Get quote for instrument
  POST /market-data/historical - Get historical bars

Account:
  GET  /account/summary     - Account summary (NLV, margin)
  GET  /account/positions   - Open positions
  GET  /account/pnl         - Profit & loss

Orders:
  POST /orders/preview           - Preview order impact
  POST /orders                   - Place order
  GET  /orders/open              - List open orders
  GET  /orders/{id}/status       - Get order status
  POST /orders/{id}/cancel       - Cancel order

Documentation:
  GET  /docs    - Swagger UI
  GET  /redoc   - ReDoc



## Summary

Phase 5 provides a complete REST API layer for the IBKR Gateway.

### Key Features:
- **FastAPI Framework** - Modern async Python web framework
- **OpenAPI Documentation** - Auto-generated Swagger/ReDoc
- **Pydantic Validation** - Strong request/response typing
- **API Key Auth** - Optional X-API-Key header authentication
- **Consistent Errors** - Structured error responses with codes
- **CORS Support** - Cross-origin requests for web clients

### Running the Server:
```bash
# Basic
uvicorn api.server:app

# With auto-reload for development
uvicorn api.server:app --reload

# With custom port
uvicorn api.server:app --port 8080

# With authentication
API_KEY=your-secret uvicorn api.server:app
```

### Environment Variables:
- `TRADING_MODE` - "paper" or "live"
- `ORDERS_ENABLED` - "true" or "false"
- `API_KEY` - Optional API key for authentication
- `API_REQUEST_TIMEOUT` - Request timeout in seconds (default: 30)
- `IBKR_GATEWAY_HOST` - IBKR gateway host (default: 127.0.0.1)
- `IBKR_GATEWAY_PORT` - IBKR gateway port (default: 4002)